In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier, _tree
import logging

# Set up logging for professional audit trails
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

class EnterpriseDecisionEngine:
    def __init__(self, max_depth=3):
        self.max_depth = max_depth
        self.model = DecisionTreeClassifier(max_depth=self.max_depth, random_state=42)
        self.rules = []
        self.feature_names = ['Income_k', 'Credit_Score', 'Employment_Years']
        self.class_names = ['REJECTED', 'APPROVED']

    def generate_training_data(self):
        """Generates synthetic historical loan data."""
        np.random.seed(42)
        n = 1000
        income = np.random.randint(20, 150, n)
        credit = np.random.randint(300, 850, n)
        emp = np.random.randint(0, 20, n)

        # Internal Logic: High credit OR High income with moderate employment
        target = ((credit > 680) | (income > 90) & (emp > 2)).astype(int)

        return pd.DataFrame({
            'Income_k': income, 'Credit_Score': credit,
            'Employment_Years': emp, 'Loan_Status': target
        })

    def build_knowledge_base(self):
        """Trains model and extracts the 'Knowledge Base' of rules."""
        try:
            df = self.generate_training_data()
            X = df[self.feature_names]
            y = df['Loan_Status']

            self.model.fit(X, y)

            # Recursive tree traversal
            tree_ = self.model.tree_
            def recurse(node, current_conditions):
                if tree_.feature[node] != _tree.TREE_UNDEFINED:
                    name = self.feature_names[tree_.feature[node]]
                    threshold = tree_.threshold[node]

                    # Left: Condition is True (<=)
                    recurse(tree_.children_left[node], current_conditions + [(name, "<=", round(threshold, 2))])
                    # Right: Condition is False (>)
                    recurse(tree_.children_right[node], current_conditions + [(name, ">", round(threshold, 2))])
                else:
                    val = tree_.value[node][0]
                    self.rules.append({
                        'conditions': current_conditions,
                        'decision': self.class_names[np.argmax(val)],
                        'confidence': np.max(val) / np.sum(val),
                        'samples': int(np.sum(val))
                    })

            recurse(0, [])
            logging.info(f"Successfully extracted {len(self.rules)} business rules.")
        except Exception as e:
            logging.error(f"Failed to build knowledge base: {e}")

    def run_interface(self):
        """Continuous loop for user interaction."""
        print("\n" + "="*50)
        print("AI RULE GENERATOR: LIVE EVALUATION MODE")
        print("Type 'exit' at any prompt to stop.")
        print("="*50)

        while True:
            try:
                # 1. Collect Inputs
                raw_income = input("\nAnnual Income (k$): ").strip().lower()
                if raw_income == 'exit': break

                raw_credit = input("Credit Score (300-850): ").strip().lower()
                if raw_credit == 'exit': break

                raw_emp = input("Years of Employment: ").strip().lower()
                if raw_emp == 'exit': break

                # 2. Validate & Parse
                user_data = {
                    'Income_k': float(raw_income),
                    'Credit_Score': float(raw_credit),
                    'Employment_Years': float(raw_emp)
                }

                # 3. Match against Rule Engine
                match_found = False
                for rule in self.rules:
                    match = True
                    for feat, op, val in rule['conditions']:
                        if op == "<=" and not (user_data[feat] <= val): match = False; break
                        if op == ">" and not (user_data[feat] > val): match = False; break

                    if match:
                        print(f"\n[DECISION]: {rule['decision']}")
                        print(f"[RULE USED]: IF " + " AND ".join([f"{f} {o} {v}" for f, o, v in rule['conditions']]))
                        print(f"[STRENGTH]: {rule['confidence']:.2%} confidence based on {rule['samples']} historical cases.")
                        match_found = True
                        break

                if not match_found:
                    print("\n[RESULT]: MANUAL REVIEW REQUIRED (Input outside normal bounds)")

            except ValueError:
                print("\n>> ERROR: Please enter valid numbers or 'exit' to quit.")
            except Exception as e:
                print(f"\n>> ERROR: An unexpected issue occurred: {e}")

# --- Initialize and Run ---
if __name__ == "__main__":
    engine = EnterpriseDecisionEngine(max_depth=3)
    engine.build_knowledge_base()
    engine.run_interface()


AI RULE GENERATOR: LIVE EVALUATION MODE
Type 'exit' at any prompt to stop.

Annual Income (k$): 50000
Credit Score (300-850): 750
Years of Employment: 1

[DECISION]: APPROVED
[RULE USED]: IF Income_k > 90.5 AND Employment_Years <= 2.5 AND Credit_Score > 678.0
[STRENGTH]: 100.00% confidence based on 1 historical cases.

Annual Income (k$): 5000
Credit Score (300-850): 500
Years of Employment: 1

[DECISION]: REJECTED
[RULE USED]: IF Income_k > 90.5 AND Employment_Years <= 2.5 AND Credit_Score <= 678.0
[STRENGTH]: 100.00% confidence based on 1 historical cases.

Annual Income (k$): 300000
Credit Score (300-850): 750
Years of Employment: 2

[DECISION]: APPROVED
[RULE USED]: IF Income_k > 90.5 AND Employment_Years <= 2.5 AND Credit_Score > 678.0
[STRENGTH]: 100.00% confidence based on 1 historical cases.

Annual Income (k$): 80000
Credit Score (300-850): 540
Years of Employment: 1

[DECISION]: REJECTED
[RULE USED]: IF Income_k > 90.5 AND Employment_Years <= 2.5 AND Credit_Score <= 678.0
[S